In [8]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Quad
from datetime import datetime
from math import pi
from sklearn.preprocessing import StandardScaler

#### Things to review:


### Things to do:
- Play with more features on and off
- Look at pandas docs to understand 
- Derivatives of moving averages?
- Attempt to do this feature generation function for Keras 
- Logistic regression with polynomial features
- Keras dense neural net
- - Put the 'change' and 'diff' variable stuff into the CSV loading concatenating thing!!!
- - (this is to prevent the boundaries between data leading to bad predictions)
- Use the sklearn XGBoost 
- For generating your predictions: non-maximal suppression, how to go from the probabilities from drawing rectangles 
- - May be able to use threshholds
- For proximal and distal, work from only the subset of examples where there's a rectangle starting
- Then use a similar set of features, either just do a regression, predict the real value the price, \
- or break the target into classes like we had done before
- Set up some validation to compare the class method vs. the actual predicting of the real numbers 

#### DEFINE SUPPLY OR DEMAND ZONE

- identify first candle that has OHLC above or below proximal. if above, DEMAND, if below, SUPPLY
- if price (CLOSE of most recent or current candle) is below most recent zone then the zone is a supply zone, if price (CLOSE of most recent or current candle) is above most recent zone then it's a demand zone 

##### Logic pseudocode
```
for each Rectangle
    if H and L of any future candle, starting from current Rectangle epoch > Proximal
    zone = DEMAND

    if H and L of any future candle, starting from current Rectangle epoch < Proximal
    zone = SUPPLY

AND 

    if Close of any future candle, starting from current Rectangle epoch > Proximal AND Distal
        zone = DEMAND

    if Close of any future candle, starting from current Rectangle epoch < Proximal AND Distal
        zone = SUPPLY
```

##### Python pseudocode

```

for row in df_ohlc_and_rectangles:
    if row['Object Name'] != NaN:
        if any_following_row['H'] & any_following_row['L'] > row['Proximal']:
            if any_following_row['C'] > row['Proximal'] AND row['Distal']:
                row['Supply/Demand'] = 'DEMAND'
                
        elif any_following_row['H'] & any_following_row['L'] < row['Proximal']:
            if any_following_row['C'] < row['Proximal'] AND row['Distal']:
                row['Supply/Demand'] = 'SUPPLY'
    else:
        continue
    

```

In [9]:
def supply_or_demand(df):
    
    # Set all Supply/Demand row to nans to start clean
    df_ohlc_and_rectangles['Supply/Demand'] = np.nan
    # Create list of all zones, filtered by if there's a proximal in the row
    valid_proximal = df[df.Proximal.isnull() == False]
    
    # Create list of the indices of the zones
    proximal_indices = valid_proximal.index.tolist()
    
    # For each zone
    for idx in proximal_indices:

        # If current iterable index + 1 is > len (e.g. if we are at the end of the DF)
        if idx+1 > len(df):

            # Quit the loop
            break

        # Create a dataframe with the rows to look through, which is starting from current
        df_following_rows = df.iloc[idx+1:]

        # Test if it's a demand zone and create a series of valid potential demand flags
        test_demand = ((df_following_rows.H > df.iloc[idx]['Proximal']) & (df_following_rows.L > df.iloc[idx]['Proximal'])
                                                               & (df_following_rows.C > df.iloc[idx]['Proximal'])
                                                               & (df_following_rows.C > df.iloc[idx]['Distal']))

        # Test if it's a supply zone and create a series of potential supply flags
        test_supply = ((df_following_rows.H < df.iloc[idx]['Proximal']) & (df_following_rows.L < df.iloc[idx]['Proximal'])
                                                               & (df_following_rows.C < df.iloc[idx]['Proximal'])
                                                                & (df_following_rows.C < df.iloc[idx]['Distal']))

        # Store only the instances of areas that could indicate a demand zone
        demand = test_demand[test_demand == True]

        # Store only the instances of areas that could indicate a supply zone
        supply = test_supply[test_supply == True]

        # Store the very first item in the demand list
        demand_index = demand.index[0] if len(demand) > 0 else 0
        # Store the very first item in the supply list    
        supply_index = supply.index[0] if len(supply) > 0 else 0

        # If demand_index is earlier in the list than supply, then it's a demand zone
        if (demand_index < supply_index):
            df.ix[idx, 'Supply/Demand'] = 'DEMAND'
        else:
            df.ix[idx, 'Supply/Demand'] = 'SUPPLY'

#### Define/correct the 'Date End' for a rectangle

pseudocode
```
for each Rectangle  
    if DEMAND:  
        if L of any future candle, starting from current Rectangle epoch, is below Distal of current candle, \  
        zone end is at the epoch of the candle which eliminated it  
    if SUPPLY:  
        if H of any future candle, starting from current Rectangle epoch, is above Distal of current candle, \  
        zone end is at the epoch of the candle which eliminated it  
```
Python-esque pseudocode
```
for row in df_ohlc_and_rectangles:
    if row['Supply/Demand'] = 'DEMAND':
        if '''any following_row['L']''' < row['Distal']:
            row['Epoch Date End'] = following_row['Epoch Date']
    elif row['Supply/Demand'] = 'SUPPLY':
        if '''any following_row['H']''' > row['Distal']:
            row['Epoch Date End'] = following_row['Epoch Date']
    else:
        continue
```

In [10]:
def zone_ender(df):

    # Create list of all zones, filtered by if there's a proximal in the row
    valid_proximal = df[df.Proximal.isnull() == False]
    
    # Create list of the indices of the zones
    proximal_indices = valid_proximal.index.tolist()
    
    # For each zone
    for idx in proximal_indices:

        # If current iterable index + 1 is > len (e.g. if we are at the end of the DF)
        if idx+1 > len(df):
            # Quit the loop
            break

        # Create a dataframe with the rows to look through, which is starting from current
        df_following_rows = df.iloc[idx+1:]
        
        if df.ix[idx, 'Supply/Demand'] == 'DEMAND':
            # Create a series of potential end flags
            test_demand_end = ((df_following_rows.L < df.iloc[idx]['Distal']))
            # Store only the rows that could be where the zone ends
            demand_end = test_demand_end[test_demand_end == True]
            # Store the very first item in the demand list
            demand_end_index = demand_end.index[0] if len(demand_end) > 0 else 0
            
            # Mark the current iterable Epoch Date end as the Epoch Date of the zone that ended it
            df.ix[idx, 'Epoch Date End'] = df.iloc[demand_end_index]['Epoch Date']
            #print("the df.iloc[demand_end_index]['Epoch Date'] is",  df.iloc[demand_end_index]['Epoch Date'])
        
        elif df.ix[idx, 'Supply/Demand'] == 'SUPPLY': 
            # Create a series of potential end flags
            test_supply_end = ((df_following_rows.H > df.iloc[idx]['Distal']))
            # Store only the rows that could be where the zone ends
            supply_end = test_supply_end[test_supply_end == True]
            # Store the very first item in the supply list    
            supply_end_index = supply_end.index[0] if len(supply_end) > 0 else 0

            df.ix[idx, 'Epoch Date End'] = df.iloc[supply_end_index]['Epoch Date']
            #print("the df.iloc[supply_end_index]['Epoch Date'] is",  df.iloc[supply_end_index]['Epoch Date'])
        else:
            pass

In [24]:
# Create list of target csvs
data = ["EURJPY1440.csv", "EURGBP1440.csv","USDCAD1440.csv","GBPUSD1440.csv",\
        "EURUSD14402.csv","USDCHF1440.csv","EURAUD14402.csv","AUDUSD1440.csv",\
        'indi_EURCAD_15.csv', 'indi_USDCAD_15.csv', 'indi_USDJPY_15.csv', 'indi_USDJPY_60.csv']

In [ ]:
'''
def generate_features(dftrain):
    dftrain["change"] = dftrain["O"] - dftrain["C"]
    dftrain["range"] = dftrain["H"] - dftrain["L"]
    # allows us to ignore channels
    channels = ["O", "C"] # ["O", "H", "L", "change", "range"]
    features = []

    # shifts: value at time +/- n
    # days ahead (should be negative), days before (should be positive)
    features.append(pd.concat([dftrain[["change"]].shift(i).add_suffix("_change_{}".format(i)) for i in range(-4, 6)], axis=1))

    # diffs: value today - value at today +/- n
    features.append(pd.concat([dftrain[channels].diff(i).add_suffix("_diff_{}".format(i)) for i in range(-4, 6)], axis=1))

    # second differences: change in value today - change in value at today +/- n
    #features.append(pd.concat([
    # dftrain[channels].diff().diff(i) for i in range(-4, 4)
    #], axis=1))

    # diff shifts: change in value at today +/- n
    #features.append(pd.concat([
    # dftrain[channels].diff().shift(i) for i in range(-4, 4)
    #], axis=1))

    # Exponenential moving averages:
    # features.append(pd.concat([
    # pd.ewma(dftrain[channels], span=10).shift(i) for i in range(-1,2)
    #], axis=1))
    #features.append(pd.concat([
    # pd.ewma(dftrain[channels], span=10).diff(i) for i in range(-1, 2)
    #], axis=1))

    # Exponenential moving std deviation:
    #features.append(pd.ewmstd(dftrain[channels], span=10))
    
    return pd.concat(features, axis=1), dftrain['rectangle_here']
'''

In [40]:
def prepare_data(ohlc_data):
    # Make sure df_ohlc_and_rectangles is an empty df
    df_ohlc_and_rectangles = pd.DataFrame(data=None)
    
    # Loop to pull in parse and clean up the data 
    for each in ohlc_data:
        print(each)
        # load ohlc, label columns
        df_ohlc = pd.read_csv(each, usecols=['Date', 'O', 'H', 'L', 'C', 'V', 'Supply/Demand'])
        df_ohlc.columns = ['Epoch Date', 'O', 'H', 'L', 'C', 'V', 'Supply/Demand']

        # Scale values based on the movement within that particular currency pair    
        print("Values before scaling:") # Print values before scaling 
        print(df_ohlc[['O','H','L','C','V' ]][0:5])

        # scaling the dataset and print the first 5 rows after scaling
        scaler = StandardScaler()
        df_ohlc[['O','H','L','C','V' ]] = scaler.fit_transform(df_ohlc[['O','H','L','C','V' ]])
        print("Scaled data:")
        print(df_ohlc[['O','H','L','C','V' ]][0:5])

        # inverse transform and print the first 5 rows to ensure scaling happened properly
        print("Testing data de-scaling:")
        inversed = scaler.inverse_transform(df_ohlc[['O','H','L','C','V' ]])
        for i in range(5):
            print(inversed[i])

        # load rects, label columns    
        df_rects = pd.read_csv(each, usecols=['Object Name', 'Date Start', 'Date End', 'Proximal', 'Distal']) 
        df_rects.columns = ['Object Name', 'Epoch Date Start', 'Epoch Date End', 'Proximal', 'Distal']
        df_rects = df_rects.dropna()    

        # Merge and align them together, using the Date Start as the point to merge them
        df_merged = df_ohlc.merge(df_rects, left_on='Epoch Date', right_on='Epoch Date Start', how='left')
        print("Merging:", each, df_merged.shape)

        # run supply or demand zone func
        supply_or_demand(df_merged)
        zone_ender(df_merged)

        # Generate new features
        print("Generating features:", each, " Difference between open and close")
        df_merged["change"] = df_merged["O"] - df_merged["C"] # add change column, difference between open and close

        print("Generating features:", each, " Range between high and low")
        df_merged["range"] = df_merged["H"] - df_merged["L"] # add range, difference between high and low

        # allows us to easily ignore channels when testing different feature generation 
        channels = ["O", "C"] # ["O", "H", "L", "change", "range"]
        features = []

        # allows us to easily modify how many days ahead and before we look:
        lookback = 6 # how many days we look back, should be positive
        lookforward = -4 # how many days we look forward, should be negative

        for row in each:
            # shifts: value at time +/- n    
            features.append(pd.concat([df_merged[["change"]].shift(i).add_suffix(
                "_change_{}".format(i)) for i in range(lookforward, lookback)], axis=1))

            # diffs: value today - value at today +/- n
            features.append(pd.concat([df_merged[channels].diff(i).add_suffix(
                "_diff_{}".format(i)) for i in range(lookforward, lookback)], axis=1))
            df_merged = pd.concat([df_merged, pd.DataFrame(features)])
        

        # Merge all the loaded data together
        df_ohlc_and_rectangles = pd.concat([df_merged, df_ohlc_and_rectangles])

        print("Finished:", each, df_merged.shape)
        print("--------------------------------")

    return df_ohlc_and_rectangles

In [41]:
test_data = ["EURJPY1440.csv", "EURGBP1440.csv"]

In [42]:
prepare_data(test_data)

EURJPY1440.csv
Values before scaling:
        O       H       L       C    V
0  141.55  143.48  140.86  143.30  350
1  143.29  144.85  142.99  144.54  248
2  144.58  144.92  143.71  143.89  174
3  143.92  144.40  143.50  144.10  163
4  144.14  144.79  143.69  144.65  170
Scaled data:
          O         H         L         C         V
0  0.767995  0.837862  0.772362  0.867861 -0.765682
1  0.867430  0.916201  0.894174  0.938772 -0.766932
2  0.941149  0.920203  0.935350  0.901601 -0.767840
3  0.903433  0.890469  0.923341  0.913610 -0.767975
4  0.916005  0.912770  0.934207  0.945063 -0.767889
Testing data de-scaling:
[ 141.55  143.48  140.86  143.3   350.  ]
[ 143.29  144.85  142.99  144.54  248.  ]
[ 144.58  144.92  143.71  143.89  174.  ]
[ 143.92  144.4   143.5   144.1   163.  ]
[ 144.14  144.79  143.69  144.65  170.  ]
Merging: EURJPY1440.csv (4950, 12)
Generating features: EURJPY1440.csv  Difference between open and close
Generating features: EURJPY1440.csv  Range between high and lo

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  result = result.union(other)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  result = result.union(other)


Finished: EURJPY1440.csv (5160, 15)
--------------------------------
EURGBP1440.csv
Values before scaling:
        O       H       L       C    V
0  0.6552  0.6556  0.6519  0.6533   56
1  0.6526  0.6559  0.6515  0.6541   74
2  0.6544  0.6547  0.6498  0.6519   74
3  0.6524  0.6538  0.6509  0.6517   52
4  0.6517  0.6612  0.6516  0.6592  118
Scaled data:
          O         H         L         C         V
0 -0.933431 -0.959714 -0.942264 -0.956871 -0.748744
1 -0.961353 -0.956508 -0.946582 -0.948265 -0.748410
2 -0.942023 -0.969331 -0.964935 -0.971932 -0.748410
3 -0.963501 -0.978947 -0.953060 -0.974084 -0.748818
4 -0.971019 -0.899877 -0.945502 -0.893400 -0.747596
Testing data de-scaling:
[  0.6552   0.6556   0.6519   0.6533  56.    ]
[  0.6526   0.6559   0.6515   0.6541  74.    ]
[  0.6544   0.6547   0.6498   0.6519  74.    ]
[  0.6524   0.6538   0.6509   0.6517  52.    ]
[   0.6517    0.6612    0.6516    0.6592  118.    ]
Merging: EURGBP1440.csv (4951, 12)
Generating features: EURGBP1440.cs

,Epoch Date,O,H,L,C,V,Supply/Demand,Object Name,Epoch Date Start,Epoch Date End,Proximal,Distal,change,range,0
0,893635200.0,-0.933431,-0.959714,-0.942264,-0.956871,-0.748744,NaN,NaN,NaN,NaN,NaN,NaN,0.023440,-0.017450,NaN
1,893721600.0,-0.961353,-0.956508,-0.946582,-0.948265,-0.748410,NaN,NaN,NaN,NaN,NaN,NaN,-0.013089,-0.009926,NaN
2,893808000.0,-0.942023,-0.969331,-0.964935,-0.971932,-0.748410,NaN,NaN,NaN,NaN,NaN,NaN,0.029909,-0.004395,NaN
3,893894400.0,-0.963501,-0.978947,-0.953060,-0.974084,-0.748818,SUPPLY,Rectangle 23750,893894400.0,1.208218e+09,0.6524,0.6498,0.010582,-0.025888,NaN
4,893980800.0,-0.971019,-0.899877,-0.945502,-0.893400,-0.747596,NaN,NaN,NaN,NaN,NaN,NaN,-0.077619,0.045625,NaN
5,894240000.0,-0.914101,-0.912699,-0.898000,-0.890172,-0.748744,NaN,NaN,NaN,NaN,NaN,NaN,-0.023929,-0.014699,NaN
6,894326400.0,-0.887253,-0.844314,-0.874249,-0.822398,-0.747892,NaN,NaN,NaN,NaN,NaN,NaN,-0.064855,0.029935,NaN
7,894412800.0,-0.831409,-0.826149,-0.818110,-0.808412,-0.748466,NaN,NaN,NaN,NaN,NaN,NaN,-0.022996,-0.008039,NaN
8,894499200.0,-0.807783,-0.759901,-0.801916,-0.770760,-0.746967,NaN,NaN,NaN,NaN,NaN,NaN,-0.037023,0.042015,NaN
9,894585600.0,-0.760530,-0.735325,-0.743618,-0.737411,-0.747874,NaN,NaN,NaN,NaN,NaN,NaN,-0.023120,0.008293,NaN


In [38]:
df_ohlc_and_rectangles[0:5]

,Supply/Demand


In [39]:
df_ohlc_and_rectangles.shape

(0, 1)

In [ ]:
df_ohlc_and_rectangles.to_csv("df_ohlc_and_rectangles_checkpoint-05.14.17-with-v.csv")

### Checkpoint
- Load in the processed data from above into one CSV

In [22]:
df_ohlc_and_rectangles = pd.read_csv("df_ohlc_and_rectangles_checkpoint-05.14.17-with-v.csv")

In [23]:
df_ohlc_and_rectangles[0:20]

,Unnamed: 0,C,Distal,Epoch Date,Epoch Date End,Epoch Date Start,H,L,O,Object Name,Proximal,Supply/Demand,V
0,0,-0.777520,NaN,893376000.0,NaN,NaN,-0.779661,-0.769033,-0.761712,NaN,NaN,NaN,-0.783529
1,1,-0.799822,NaN,893635200.0,NaN,NaN,-0.806972,-0.785654,-0.784000,NaN,NaN,NaN,-0.784301
2,2,-0.825948,NaN,893721600.0,NaN,NaN,-0.824756,-0.802914,-0.801194,NaN,NaN,NaN,-0.784665
3,3,-0.788353,NaN,893808000.0,NaN,NaN,-0.808242,-0.806110,-0.827304,NaN,NaN,NaN,-0.783847
4,4,-0.787715,NaN,893894400.0,NaN,NaN,-0.786013,-0.768394,-0.789731,NaN,NaN,NaN,-0.783620
5,5,-0.792176,0.6546,893980800.0,1.184803e+09,893980800.0,-0.780931,-0.774786,-0.787821,Rectangle 21197,0.6488,SUPPLY,-0.782801
6,6,-0.836143,NaN,894240000.0,NaN,NaN,-0.816499,-0.820813,-0.796736,NaN,NaN,NaN,-0.783324
7,7,-0.840604,NaN,894326400.0,NaN,NaN,-0.853337,-0.826566,-0.838130,NaN,NaN,NaN,-0.784233
8,8,-0.864818,NaN,894412800.0,NaN,NaN,-0.855878,-0.847023,-0.842587,NaN,NaN,NaN,-0.783665
9,9,-0.876925,NaN,894499200.0,NaN,NaN,-0.881283,-0.861086,-0.866150,NaN,NaN,NaN,-0.784074


### Bokeh Data Visualization
- NOTE: This does not work with the massive dataset because all the candles are on top of each other

df = df_ohlc_and_rectangles
#### Convert Dates to datetime
df["Epoch Date"] = pd.to_datetime(df["Epoch Date"], unit='s')
df["Epoch Date Start"] = pd.to_datetime(df["Epoch Date Start"], unit='s')
df["Epoch Date End"] = pd.to_datetime(df["Epoch Date End"], unit='s')

w = 12*60*60*1000 # half day in ms

TOOLS = "pan,xwheel_zoom,wheel_zoom,box_zoom,reset,save"
p = figure(x_axis_type="datetime", tools=TOOLS, plot_height=500, 
           plot_width=1200, title="Candlesticks and zones test 1", active_scroll='xwheel_zoom')
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.5

p.segment(df['Epoch Date'], df.H, df['Epoch Date'], df.L, color="black")
p.vbar(df['Epoch Date'], w, df.O, df.C)

#### Add Rectangle data
df_sp = df.dropna(subset=['Supply/Demand'])

source_supply = ColumnDataSource(df_sp.loc[df_sp['Supply/Demand']=='SUPPLY', :])
rect_supply = Quad(left="Epoch Date Start", top="Proximal", right="Epoch Date End", bottom="Distal", fill_color="firebrick", line_alpha=0.2, fill_alpha=0.2) 
p.add_glyph(source_supply, rect_supply)

source_demand = ColumnDataSource(df_sp.loc[df_sp['Supply/Demand']=='DEMAND', :])
rect_demand = Quad(left="Epoch Date Start", top="Proximal", right="Epoch Date End", bottom="Distal", fill_color="steelblue", line_alpha=0.2, fill_alpha=0.2)
p.add_glyph(source_demand, rect_demand)


output_file("candlestick.html", title="candlestick data SPC test")
show(p)  # open a browser

### First munging data to multi-hot encode attempt

In [15]:
# df_ohlc_and_rectangles["rectangle_here"] = df_ohlc_and_rectangles["Epoch Date Start"].notnull()

In [16]:
# skipping the mapping to integer labels and going to one-hot encoding straight off
# integer labels can be reconstructed if needed.
'''
for column in ["Proximal", "Distal"]:
    df_ohlc_and_rectangles["{}_yesterday_open".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["O"].shift())
    df_ohlc_and_rectangles["{}_yesterday_close".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["C"].shift())
    df_ohlc_and_rectangles["{}_yesterday_high".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["H"].shift())
    df_ohlc_and_rectangles["{}_yesterday_low".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["L"].shift())
    df_ohlc_and_rectangles["{}_today_open".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["O"])
    df_ohlc_and_rectangles["{}_today_close".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["C"])
    df_ohlc_and_rectangles["{}_today_high".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["H"])

df_ohlc_and_rectangles
'''

'\nfor column in ["Proximal", "Distal"]:\n    df_ohlc_and_rectangles["{}_yesterday_open".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["O"].shift())\n    df_ohlc_and_rectangles["{}_yesterday_close".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["C"].shift())\n    df_ohlc_and_rectangles["{}_yesterday_high".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["H"].shift())\n    df_ohlc_and_rectangles["{}_yesterday_low".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["L"].shift())\n    df_ohlc_and_rectangles["{}_today_open".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["O"])\n    df_ohlc_and_rectangles["{}_today_close".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["C"])\n    df_ohlc_and_rectangles["{}_today_high".format(column)] = (df_ohlc_and_rectangles[column] == df_ohlc_and_rectangles["H"])\n\ndf_ohlc_and_rectangles\n'

In [17]:
'''
# We have three separate targets:
y_rectangle = df_ohlc_and_rectangles["rectangle_here"]
y_proximal = df_ohlc_and_rectangles.loc[:, df_ohlc_and_rectangles.columns.str.startswith("Proximal_")]
y_distal = df_ohlc_and_rectangles.loc[:, df_ohlc_and_rectangles.columns.str.startswith("Distal_")]
'''

'\n# We have three separate targets:\ny_rectangle = df_ohlc_and_rectangles["rectangle_here"]\ny_proximal = df_ohlc_and_rectangles.loc[:, df_ohlc_and_rectangles.columns.str.startswith("Proximal_")]\ny_distal = df_ohlc_and_rectangles.loc[:, df_ohlc_and_rectangles.columns.str.startswith("Distal_")]\n'

In [18]:
'''
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# have a look at the distal/proximal class distribution
y_distal.sum().plot.bar()
plt.show()
y_proximal.sum().plot.bar()
plt.show()

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-18-9ec53a54d11a>, line 9)

### Predict zones with Keras

- Ignore Epoch End, if it's possible to label mathematically
- Ignore Volume if it's not important
- LSTM for now? 


##### First make it a binary classifier problem, should there be a rectangle here?
- Convert 'Epoch Date Start' for rectangles to 0 or 1, either a rectangle exists or does not exist at that point
- 'Epoch Date End' is determined mathematically, or is NaN

##### Then one hot encode to booleans the Proximal based on whether its related to a number +/- 6 days in the future
- Break the 'Proximal' and 'Distal' prediction down into classes:
```
0 for Yesterday Open
1 for Yesterday Close
2 for Yesterday H
3 for Yesterday L
4 for Today O
5 for Today C
6 for Today H
```

In [19]:
# Remove the nan rows
#df_ohlc_and_rectangles_dropna = df_ohlc_and_rectangles.dropna(subset=['O', 'H', 'L', 'C'])

In [24]:
# Create "labels" 
df_ohlc_and_rectangles["rectangle_here"] = df_ohlc_and_rectangles["Epoch Date Start"].notnull()
y_rectangle = df_ohlc_and_rectangles["rectangle_here"]

### Prepare the data in Pandas for time-series analysis

In [ ]:
multiple_timestamp_data = []
for i in range(-6, 2):
    multiple_timestamp_data.append(X.shift(i).add_suffix("_{}".format(i)))

In [ ]:
X_mult = pd.concat(multiple_timestamp_data, axis=1)

### Prepare the data for Keras in numpy

In [43]:
# For each row, combine 6 rows before and 2 rows ahead into the same line.
# This is because this is the maximum range that includes data which is valuable to defining a zone 
df_data = df_ohlc_and_rectangles[["O", "H", "L", "C", "V"]]
multiple_timestamp_O = []
multiple_timestamp_H = []
multiple_timestamp_L = []
multiple_timestamp_C = []
multiple_timestamp_V = []

# days ahead (should be negative), days before (should be positive)
for i in range(-6, 2):
    multiple_timestamp_O.append(df_data["O"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_H.append(df_data["H"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_L.append(df_data["L"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_C.append(df_data["C"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_V.append(df_data["V"].shift(i).add_suffix("_{}".format(i)))

In [44]:
keras_data_O = np.array(multiple_timestamp_O).T.reshape(-1,8,1)
keras_data_H = np.array(multiple_timestamp_H).T.reshape(-1,8,1)
keras_data_C = np.array(multiple_timestamp_C).T.reshape(-1,8,1)
keras_data_L = np.array(multiple_timestamp_L).T.reshape(-1,8,1)
keras_data_V = np.array(multiple_timestamp_V).T.reshape(-1,8,1)

In [45]:
keras_data_C.shape

(38771, 8, 1)

In [46]:
keras_data = np.concatenate([keras_data_O, keras_data_H, keras_data_L, keras_data_C, keras_data_V], axis=2)

### A big mess of data munging to remove nans

In [47]:
keras_data.shape

(38771, 8, 5)

In [48]:
df_keras_dfdf = pd.DataFrame(keras_data[1])

In [49]:
df_keras_dfdf

,0,1,2,3,4
0,-0.838130,-0.853337,-0.826566,-0.840604,-0.784233
1,-0.796736,-0.816499,-0.820813,-0.836143,-0.783324
2,-0.787821,-0.780931,-0.774786,-0.792176,-0.782801
3,-0.789731,-0.786013,-0.768394,-0.787715,-0.783620
4,-0.827304,-0.808242,-0.806110,-0.788353,-0.783847
5,-0.801194,-0.824756,-0.802914,-0.825948,-0.784665
6,-0.784000,-0.806972,-0.785654,-0.799822,-0.784301
7,-0.761712,-0.779661,-0.769033,-0.777520,-0.783529


In [50]:
keras_y_rect = y_rectangle

In [80]:
y_rectangle.shape

(38771,)

In [52]:
np_keras_y_rect = np.array(keras_y_rect) 

In [53]:
np_keras_y_rect.shape

(38771,)

#### Omg why didn't i just delete the first and last few rows the first time

In [54]:
# Drop from the train data 
np_keras_data_drop = keras_data[2:]
np_keras_data_drop = np_keras_data_drop[:-7]

In [55]:
np_keras_data_drop.shape

(38762, 8, 5)

In [56]:
# Drop from the train targets
np_keras_y_rect_drop = np_keras_y_rect[2:]
np_keras_y_rect_drop = np_keras_y_rect_drop[:-7]

In [57]:
np_keras_y_rect_drop.shape

(38762,)

In [239]:
#np_keras_y_rect_reshape = np_keras_y_rect.reshape(-1,1)

In [272]:
#np_keras_y_rect_reshape.shape

In [164]:
#np_data_and_labels = np.concatenate((keras_data, np_keras_y_rect_reshape), axis=1)

In [271]:
#np_data_and_labels

In [273]:
#np_data_and_labels_no_nan = np_data_and_labels[~np.isnan(np_data_and_labels).any(axis=1)]

In [274]:
#np_data_and_labels_no_nan.shape

In [275]:
#df_data_and_labels_no_nan = pd.DataFrame(np_data_and_labels_no_nan)

In [276]:
#df_data_and_labels_no_nan[0:10]

In [26]:
#df_y_labels = df_data_and_labels_no_nan[40]

In [278]:
#df_y_labels.shape

In [279]:
#df_data_and_labels_no_nan.shape

In [29]:
#df_ohlc_and_rectangle_data = df_data_and_labels_no_nan.drop([40],axis=1)

In [277]:
#df_ohlc_and_rectangle_data[0:5]

In [31]:
#X = np.array(df_ohlc_and_rectangle_data)

In [32]:
#y = np.array(df_y_labels)

In [33]:
#Y = Y.reshape(-1,1)

In [58]:
X = np_keras_data_drop

In [59]:
y = np_keras_y_rect_drop

In [60]:
# Show how unbalanced we are!
np.bincount(y_train.astype(int))

NameError: name 'y_train' is not defined

#### Final processed DATA and LABELS

In [314]:
#df_ohlc_and_rectangle_data.shape

In [315]:
#df_y_labels.shape

In [68]:
X.shape

(38762, 8, 5)

In [69]:
y_int = y.astype(int)

In [ ]:
# TODO: Add GroupKFold to do test/train/cross val split

# PANDAS: Rolling max / rolling min ???

### Keras sdjfkjdslkgaj

In [34]:
from keras.layers import LSTM, Dense, Input, Dropout, Conv1D
from keras.models import Model
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam, RMSprop
from keras import regularizers

Using TensorFlow backend.


In [35]:
# optimizers

sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)
adam = Adam(lr=0.01)
rmsprop = RMSprop(lr=0.01)

In [64]:
X_train = X[:-1000]
y_train = y[:-1000]
X_test = X[-1000:]
y_test = y[-1000:]

## Improvements
- At this point it looks like a problem with the data
- Feature engineering on forests

In [38]:
# Joe's model, modified 
def create_model():
    # create model
    model = Sequential()
    model.add(LSTM(64, input_shape=(8,5), activation='relu', inner_activation='hard_sigmoid'))
    #model.add(Dropout(0.1))
    #model.add(LSTM(64, kernel_regularizer=regularizers.l2(0.01), activation='relu', ))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [71]:
# Stackoverflow / Keras examples / TW hax
def create_model():
    model = Sequential()
    model.add(LSTM(32, 
                   input_shape=(8,5), 
                   return_sequences=True, 
                   activation='relu'
                  )
             )
    model.add(Dropout(0.2))
    model.add(LSTM(32,
                    return_sequences=False,
                    activation='relu'
                  )
             )
    model.add(Dense(1,activation='sigmoid')) 
    model.compile(loss='binary_crossentropy', optimizer=rmsprop, metrics=['accuracy'])
    return model

In [458]:
from sklearn.utils import class_weight

# In order to calculate the class weight 
class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [465]:
class_weight

array([ 0.57330646,  3.91034048])

In [39]:
model = create_model()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(64, recurrent_activation="hard_sigmoid", activation="sigmoid", input_shape=(8, 5))`
  """


In [128]:
model.fit(X_train, y_train, 
          epochs=1, 
          class_weight={0:1, 1:7},
          #validation_data=(X_val,y_val)
         )

IndentationError: unexpected indent (<ipython-input-128-fd223a087018>, line 2)

In [ ]:
AUC_PR = average_precision_score(y_true=y_test, y_score=model.predict(X_test), average='weighted')

In [ ]:
# to investigate- rocauc -- 

### Trees!
- TODO: Ensemble of decision trees XGBoost
- Must use the 2D data not the 3D data

In [213]:
def generate_features(dftrain):
    dftrain["change"] = dftrain["O"] - dftrain["C"]
    dftrain["range"] = dftrain["H"] - dftrain["L"]
    # allows us to ignore channels
    channels = ["O", "C"] # ["O", "H", "L", "change", "range"]
    features = []

    # shifts: value at time +/- n
    # days ahead (should be negative), days before (should be positive)
    features.append(pd.concat([dftrain[["change"]].shift(i).add_suffix("_change_{}".format(i)) for i in range(-4, 6)], axis=1))

    # diffs: value today - value at today +/- n
    features.append(pd.concat([dftrain[channels].diff(i).add_suffix("_diff_{}".format(i)) for i in range(-4, 6)], axis=1))

    # second differences: change in value today - change in value at today +/- n
    #features.append(pd.concat([
    # dftrain[channels].diff().diff(i) for i in range(-4, 4)
    #], axis=1))

    # diff shifts: change in value at today +/- n
    #features.append(pd.concat([
    # dftrain[channels].diff().shift(i) for i in range(-4, 4)
    #], axis=1))

    # Exponenential moving averages:
    # features.append(pd.concat([
    # pd.ewma(dftrain[channels], span=10).shift(i) for i in range(-1,2)
    #], axis=1))
    #features.append(pd.concat([
    # pd.ewma(dftrain[channels], span=10).diff(i) for i in range(-1, 2)
    #], axis=1))

    # Exponenential moving std deviation:
    #features.append(pd.ewmstd(dftrain[channels], span=10))
    
    return pd.concat(features, axis=1), dftrain['rectangle_here']

In [214]:
df_X, df_y = generate_features(df_ohlc_and_rectangles)

In [215]:
# For each row, combine 6 rows before and 2 rows ahead into the same line.
# This is because this is the maximum range that includes data which is valuable to defining a zone 
df_data = df_ohlc_and_rectangles[["O", "H", "L", "C", "V"]]
multiple_timestamp_O = []
multiple_timestamp_H = []
multiple_timestamp_L = []
multiple_timestamp_C = []
multiple_timestamp_V = []

for i in range(-6, 2):
    multiple_timestamp_O.append(df_data["O"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_H.append(df_data["H"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_L.append(df_data["L"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_C.append(df_data["C"].shift(i).add_suffix("_{}".format(i)))
    multiple_timestamp_V.append(df_data["V"].shift(i).add_suffix("_{}".format(i)))

In [216]:
tree_data_O = np.array(multiple_timestamp_O).T
tree_data_H = np.array(multiple_timestamp_H).T
tree_data_C = np.array(multiple_timestamp_C).T
tree_data_L = np.array(multiple_timestamp_L).T
tree_data_V = np.array(multiple_timestamp_V).T

In [217]:
tree_data_O

array([[-0.79673646, -0.78782101, -0.78973147, ..., -0.78400011,
        -0.7617115 ,         nan],
       [-0.83812959, -0.79673646, -0.78782101, ..., -0.80119418,
        -0.78400011, -0.7617115 ],
       [-0.84258731, -0.83812959, -0.79673646, ..., -0.8273037 ,
        -0.80119418, -0.78400011],
       ..., 
       [        nan,         nan,         nan, ..., -0.30836567,
        -0.32568115, -0.35671187],
       [        nan,         nan,         nan, ..., -0.26173387,
        -0.30836567, -0.32568115],
       [        nan,         nan,         nan, ...,         nan,
        -0.26173387, -0.30836567]])

In [91]:
tree_data = np.concatenate([tree_data_O, tree_data_H, tree_data_C, tree_data_L, tree_data_V], axis=1)

In [92]:
# Drop from the train targets
np_tree_y_rect_drop = y_rectangle[2:]
np_tree_y_rect_drop = np_tree_y_rect_drop[:-7]

In [93]:
# Drop from the train data 
np_tree_data_drop = tree_data[2:]
np_tree_data_drop = np_tree_data_drop[:-7]

In [94]:
X_treetrain = np_tree_data_drop[:-1000]
y_treetrain = np_tree_y_rect_drop[:-1000]
X_treetest = np_tree_data_drop[-1000:]
y_treetest = np_tree_y_rect_drop[-1000:]

In [97]:
X_treetrain.shape

(37762, 40)

In [122]:
y_treetrain.shape

(37762,)

In [127]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Imputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

In [231]:
clf = make_pipeline(Imputer(), RandomForestClassifier(n_estimators=100, n_jobs=-1, 
                                                        class_weight='balanced', 
                                                        verbose=1))

In [232]:
# Joe helper model thing
def validate_model(model, X, y):
    model = model.fit(X.iloc[:-2000], y.iloc[:-2000])
    preds = model.predict_proba(X.iloc[-2000:])
    return y.iloc[-2000:], preds[:, 1]

In [233]:
y, preds = validate_model(clf, df_X, df_y)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    5.8s finished
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


In [234]:
print("Log loss:", log_loss(y, preds))
print("ROC AUC SCORE:", roc_auc_score(y, preds))

Log loss: 0.177784424908
ROC AUC SCORE: 0.953380571429


In [ ]:
# -6, 2
Log loss: 0.175330343874
ROC AUC SCORE: 0.951071244703

In [109]:
# Apply the classifier we trained to the test data
predictions = model.predict(X_treetest)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


In [152]:
predict_proba = model.predict_proba(df_X)[0:10]

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished


In [153]:
print(predict_proba)

[[ 0.93  0.07]
 [ 0.99  0.01]
 [ 0.99  0.01]
 [ 1.    0.  ]
 [ 0.97  0.03]
 [ 0.08  0.92]
 [ 1.    0.  ]
 [ 0.95  0.05]
 [ 1.    0.  ]
 [ 1.    0.  ]]


In [112]:
df_predictions = pd.DataFrame(predictions)

In [116]:
df_predictions.shape

(1000, 1)

In [117]:
y_treetest.shape

(1000,)

In [115]:
pd.concat([df_predictions, pd.DataFrame(y_treetest)], axis=1)

,0,rectangle_here
0,False,NaN
1,False,NaN
2,False,NaN
3,False,NaN
4,False,NaN
5,False,NaN
6,False,NaN
7,False,NaN
8,False,NaN
9,False,NaN


In [ ]:
# View a list of the features and their importance scores
list(zip(X_treetest, clf.feature_importances_))

In [ ]:
#### Failed attempt to use GroupKFold`
``` 
X : array-like, shape (n_samples, n_features)
Training data, where n_samples is the number of samples and n_features is the number of features.
y : array-like, shape (n_samples,)
The target variable for supervised learning problems.
```

from sklearn.model_selection import GroupKFold
X = keras_data
y = keras_y_rect
groups = np.array(len(X)/sum(range()))
group_kfold = GroupKFold(n_splits=3)
group_kfold.get_n_splits(X, y, groups)
print(group_kfold)
GroupKFold(n_splits=3)
for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, X_val, y_train, y_test, y_val)

In [428]:
print("Train shapes (x, y):", X_train.shape, y_train.shape)
print("Validation shapes (x, y):", X_val.shape, y_val.shape)
print("Test shapes (x, y):", X_test.shape, y_test.shape)

Train shapes (x, y): (31009, 8, 5) (31009,)
Validation shapes (x, y): (3876, 8, 5) (3876,)
Test shapes (x, y): (3877, 8, 5) (3877,)


In [ ]:
X_train.shape

In [ ]:
y_train.shape

### Let's try some unbalanced data stuff!!!

In [103]:
from imblearn.over_sampling import SMOTE

# Apply regular SMOTE
#kind = ['regular']
sm = SMOTE(kind='regular')
X_resampled = np.empty((0,32), float)
y_resampled = np.empty((0,1), float)
#X_res_vis = []

X_resampled, y_resampled = sm.fit_sample(X_train, y_train)
#X_resampled.append(X_res)
#y_resampled.append(y_res)
#X_res_vis.append(pca.transform(X_res))

xp = [X_resampled, y_resampled]
for i in xp:
    print(i.shape)

(50754, 40)
(50754,)


In [104]:
print("Train shapes (x, y):", X_train.shape, y_train.shape)
#print("Validation shapes (x, y):", val_x.shape, val_y.shape)
print("Test shapes (x, y):", X_test.shape, y_test.shape)
print("SMOTE Resampled shapes (x, y):", X_resampled.shape, y_resampled.shape)

Train shapes (x, y): (29073, 40) (29073,)
Test shapes (x, y): (9691, 40) (9691,)
SMOTE Resampled shapes (x, y): (50754, 40) (50754,)


### Seventh NN

In [ ]:
X_train.shape

In [ ]:
def fit_lstm(X_train, y_train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    
    # reshape training into [samples, timesteps, features]
    X = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    print("Your train data input is:", X)
    print(X.shape)
    y = y_train.reshape(-1,1)
    print("Your train data target is:", y)
    print(y.shape)
    
    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dropout(0.1))
    model.add(Dense(256, kernel_regularizer=regularizers.l2(0.01), activation='relu', ))
    model.add(Dense(y.shape[1], activity_regularizer=regularizers.l1(0.01)))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    # fit network
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=n_batch, verbose=1, shuffle=False)
        model.reset_states()
    return model

In [ ]:
# fit_lstm(data, lookback, n_seq?, batch size, number of epochs, number of neurons in first LSTM etc. )
model = fit_lstm(X_train, y_train, 1, 3, 1, 1, 1)
# Final evaluation of the model
#scores = model.evaluate(X_test, y_test, verbose=0)
#print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# reshape test data into [samples, timesteps, features]
X_test_reshape = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
#print("Your train data input is:", X)
#print(X_test_reshape.shape)
y_test_reshape = y_test.reshape(-1,1)
#print("Your train data target is:", y)
#print(y_test_reshape.shape)

In [ ]:
from sklearn import metrics

threshold = 0.5

#score = model.evaluate(X_test_reshape, y_test, batch_size=1)
#print(score)
# make predictions
expected = y_test.astype(float)
predicted = model.predict(X_test_reshape, batch_size=1)
# summarize the fit of the model
print(metrics.classification_report(expected, (predicted>threshold).astype(int)))
print(metrics.confusion_matrix(expected, (predicted>threshold).astype(int)))

In [ ]:
print(expected[0:100])
print((predicted>threshold).astype(int)[0:100])

### Run the NN

In [286]:
model = create_model()

NameError: name 'Sequential' is not defined

In [ ]:
model.fit(X_resampled, y_resampled,
          batch_size=256, epochs=50,
          #validation_data=(val_x, val_y)
         )

In [ ]:
score = model.evaluate(test_x, test_y, batch_size=128)

In [ ]:
score

In [ ]:
trainPredict = model.predict(train_x)
testPredict = model.predict(test_x)

In [ ]:
trainPredict[0:5]

In [ ]:
testPredict.shape

In [ ]:
testPredict[0:5]

### LSTM round 2
- Use the original OHLC data, not grouped data. 

In [ ]:
type(df_ohlc_and_rectangles)

In [ ]:
df_ohlc_and_rectangles[0:3]

In [ ]:
X_train = np.array(df_ohlc_and_rectangles[['O','H','L','C']])

In [ ]:
y_train = np.array(df_ohlc_and_rectangles[['rectangle_here']])

In [ ]:
X_train.shape

In [ ]:
#from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [ ]:
#max_window_length = 8
X_train = sequence.pad_sequences(X_train, maxlen=max_window_length)
#X_test = sequence.pad_sequences(X_test, maxlen=max_window_length)

In [ ]:
y_train.shape

In [ ]:
# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, nb_epoch=3, batch_size=64)